In [12]:
from keras import models
import numpy as np
from utils import min_max_scaling
from sklearn.metrics.pairwise import cosine_similarity
from score import Audio

In [8]:
encoder = models.load_model("./models/best.h5")
encoder.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 130, 560)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_25 (Conv1D)              │ (None, 130, 32)        │        53,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 65, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_26 (Conv1D)              │ (None, 65, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_16 (MaxPooling1D) │ (None, 33, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_27 (Conv1D)              │ (None, 33, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 17, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_28 (Conv1D)              │ (None, 17, 512)        │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_5          │ (None, 512)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 619,050 (2.36 MB)

 Trainable params: 619,050 (2.36 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
def mfcc_to_X(filepath):
    audio = Audio(filepath=filepath, duration=30)
    _, _, mfcc = audio.get_mfcc(80, segment_size=10)
    mfcc = np.array(mfcc)
    mfcc = mfcc.transpose(2, 1, 0)
    mfcc = np.reshape(mfcc, (130, -1))
    mfcc = np.expand_dims(mfcc, axis=-1)
    mfcc = np.expand_dims(mfcc, axis=0)
    mfcc = np.nan_to_num(mfcc, nan = 0.)
    return mfcc

def get_features(filepath, encoder: models.Model):
    mfcc = mfcc_to_X(filepath)
    res = encoder.predict(mfcc)
    res = res.flatten()
    res = min_max_scaling(res)
    return res

def get_similarity(id1, id2):
    features = [
        get_features(r".\data\music\download/" + id1 + ".mp3", encoder),
        get_features(r".\data\music\download/" + id2 + ".mp3", encoder)
    ]
    res = cosine_similarity(features)
    return np.round(res**1000 * 100, 2)[0][1] 

In [24]:
get_similarity("3eytpBOkOFA", "4n9Ie3ac6BM")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


d:\CODE\Project\Music_score\src\utils.py:214: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis = kurtosis(data, axis=2),
d:\CODE\Project\Music_score\src\utils.py:219: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew = skew(data, axis=2),


32.92